In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import cv2
import os
import glob
import PIL
from PIL import Image
img_dir = "/content/drive/My Drive/data-X/dataset/training_gallary"
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
images = []
for f1 in files:
    img = cv2.imread(f1)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #convert from RGB to BGR
    # img = img.transpose(2, 0, 1)
    images.append(img)
print(len(images))

1298


In [0]:
import os
from PIL import Image
import numpy as np
path = '/content/drive/My Drive/data-X/dataset/training_gallary'
# Store the image file names in a list as long as they are jpgs
image_names = [f for f in os.listdir(path) if os.path.splitext(f)[-1] == '.jpg']
image_names = [int(i.split('.')[0]) for i in image_names]

In [4]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/data-X/dataset/train.csv')
train_p2b = pd.read_csv('/content/drive/My Drive/data-X/dataset/train_photo_to_biz_ids.csv')
df = pd.merge(train_p2b, train, how='left',left_on='business_id',right_on='business_id')
df['labels'] = df['labels'].astype(str)
df['labels'] = df['labels'].apply(lambda x: [int(y) for y in x.split(' ') if y.isdigit()])
names = pd.DataFrame(image_names)
names = names.rename(columns={0: "image_names"})
data = pd.merge(names, df, how='left', left_on='image_names',right_on='photo_id')
labels = data['labels']
data

,image_names,photo_id,business_id,labels
0,101085,101085,2759,"[0, 3, 8]"
1,10118,10118,1801,"[1, 2, 5, 6, 8]"
2,10116,10116,3816,"[1, 2, 3, 4, 5, 6, 7, 8]"
3,101089,101089,2437,"[3, 8]"
4,10119,10119,2480,"[1, 2, 3, 4, 5, 6, 7]"
...,...,...,...,...
1293,100910,100910,2505,"[0, 1, 6, 8]"
1294,100912,100912,1717,"[1, 2, 4, 5, 6, 7]"
1295,101078,101078,501,"[0, 4, 5, 8]"
1296,101081,101081,2820,"[0, 8]"


In [0]:
validation_number = 200
image_set = {"train": images[:-200], "val": images[-200:]}
label_set = {"train": labels[:-200], "val": labels[-200:]}

In [0]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.dataset import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy

In [0]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for 
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract = True

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet34
        """
        model_ft = models.resnet34(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
 

In [0]:

def make_transform1(input_size):
    return transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

def make_transform2(input_size):
  return transforms.Compose([
      transforms.ToPILImage(),
      transforms.Resize((input_size, input_size)),
      torchvision.transforms.ToTensor(),
      transforms.Normalize((0.5,), (0.5,))
  ])



class YelpDataset(Dataset):
    def __init__(self, images, labels, label, transform):
        self.transform = transform
        Xp = []
        Xn = []
        for i, (X, y) in enumerate(zip(images, labels)):
            if label in y:
                Xp.append(transform(X))
            else:
                Xn.append(transform(X))
        length = min(len(Xn), len(Xp))
        self.X = Xp[:length] + Xn[:length]
        self.y = [1 for _ in range(length)] + [0 for _ in range(length)]
      
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

dataloader = []
for i in range(9):
    dataset = {x : YelpDataset(image_set[x], label_set[x], i, make_transform2(input_size)) for x in ['train', 'val']}
    dataloader.append({x: torch.utils.data.DataLoader(dataset[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']})


In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [11]:
# Send the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method# Send the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are 
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [12]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
m = []
for i in range(9):
  model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
  m.append(train_model(model_ft, dataloader[i], criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception")))

Epoch 0/14
----------
train Loss: 1.0880 Acc: 0.5000
val Loss: 1.0590 Acc: 0.5000

Epoch 1/14
----------
train Loss: 1.1088 Acc: 0.5000
val Loss: 1.0946 Acc: 0.5000

Epoch 2/14
----------
train Loss: 1.0929 Acc: 0.5000
val Loss: 1.0746 Acc: 0.4906

Epoch 3/14
----------
train Loss: 1.1115 Acc: 0.5000
val Loss: 1.0760 Acc: 0.5000

Epoch 4/14
----------
train Loss: 1.0942 Acc: 0.5000
val Loss: 1.0958 Acc: 0.5000

Epoch 5/14
----------
train Loss: 1.0912 Acc: 0.5000
val Loss: 1.0878 Acc: 0.5000

Epoch 6/14
----------
train Loss: 1.1013 Acc: 0.4980
val Loss: 1.1022 Acc: 0.5000

Epoch 7/14
----------
train Loss: 1.0947 Acc: 0.5000
val Loss: 1.0756 Acc: 0.5000

Epoch 8/14
----------
train Loss: 1.1005 Acc: 0.5020
val Loss: 1.1004 Acc: 0.5000

Epoch 9/14
----------
train Loss: 1.0957 Acc: 0.5000
val Loss: 1.0520 Acc: 0.5000

Epoch 10/14
----------
train Loss: 1.0949 Acc: 0.4980
val Loss: 1.1010 Acc: 0.5000

Epoch 11/14
----------
train Loss: 1.1047 Acc: 0.5020
val Loss: 1.0834 Acc: 0.5000

Ep

In [13]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from sklearn.metrics import fbeta_score

class YelpEvaluation(Dataset):
  def __init__(self, X, y, transform):
    self.X = X
    self.y = y
    self.transform = transform
  
  def __len__(self):
    return len(self.X)
  
  def __getitem__(self, idx):
    one_hot = torch.zeros(9)
    one_hot.scatter_(0, torch.tensor(self.y[idx]), 1)
    return (self.transform(self.X[idx]), one_hot)

eval_dataset = YelpEvaluation(image_set['val'], list(label_set['val']), make_transform2(input_size))
eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

def evaluate(models, eval_dataloader):
  running_ed = 0
  running_cos_sim = 0
  running_fbeta = 0
  count = 0
  for inputs, labels in eval_dataloader:
    count += 1
    preds = []
    for m, _ in models:
      m.eval()
      p = m(inputs)
      _, p = torch.max(m(inputs), dim=1)
      preds.append(p.reshape(1, -1))
    preds = torch.cat(preds, 0).transpose(0, 1)
    running_ed += nn.functional.pairwise_distance(labels, preds).mean()
    running_cos_sim += nn.functional.cosine_similarity(labels, preds).mean()
    running_fbeta += fbeta_score(labels.cpu().detach().numpy().flatten(), preds.cpu().detach().numpy().flatten(), average='macro', beta=0.5)
  print(f'euclidean_distances: {running_ed / count}' )
  print(f'cosine_similarity: {running_cos_sim / count}' )
  print(f'fbeta: {running_fbeta / count}' )

evaluate(m, eval_dataloader)






euclidean_distances: 2.266975164413452
cosine_similarity: 0.3761448562145233
fbeta: 0.417111094279377
